# Imports

In [1]:
import torch
import torchaudio
import torchaudio.transforms as T
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential, Model, callbacks
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Dropout, Bidirectional, Flatten, Dense, Input
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from jiwer import wer
from datasets import load_dataset, load_metric
from datasets import ClassLabel
import random 
import re
import pyarabic.araby as ar
import json
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, TFAutoModelForSequenceClassification, Wav2Vec2ForCTC
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import torchaudio
import librosa
import numpy as np
import string

In [2]:
#To check if GPU is detected and being used
tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Dataset Processing (Transcription)

In [3]:
dfTrain = pd.read_csv('CVtrain.tsv',sep='\t')
dfTrain["path"]
i=0
for mp3 in dfTrain["path"]:
    mp3 = mp3.replace("mp3", "wav")
    dfTrain["path"][i] = mp3 
    i=i+1

C:\anaconda\envs\nn\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
dfTest = pd.read_csv('CVtest.tsv',sep='\t')
dfTest["path"]
i=0

In [5]:
dfTest["path"]

0      common_voice_ar_19216700.mp3
1      common_voice_ar_23558552.mp3
2      common_voice_ar_23675091.mp3
3      common_voice_ar_23974261.mp3
4      common_voice_ar_23972887.mp3
                   ...             
975    common_voice_ar_19541054.mp3
976    common_voice_ar_19541055.mp3
977    common_voice_ar_19541056.mp3
978    common_voice_ar_19541057.mp3
979    common_voice_ar_19541088.mp3
Name: path, Length: 980, dtype: object

In [6]:
dfTest = pd.read_csv('CVtest.tsv',sep='\t')
dfTest["path"]
# dfTest = dfTest.dropna()

0      common_voice_ar_19216700.mp3
1      common_voice_ar_23558552.mp3
2      common_voice_ar_23675091.mp3
3      common_voice_ar_23974261.mp3
4      common_voice_ar_23972887.mp3
                   ...             
975    common_voice_ar_19541054.mp3
976    common_voice_ar_19541055.mp3
977    common_voice_ar_19541056.mp3
978    common_voice_ar_19541057.mp3
979    common_voice_ar_19541088.mp3
Name: path, Length: 980, dtype: object

In [7]:
i=0
for t in dfTest["path"]:
    t = t.replace("mp3", "wav")
    dfTest["path"][i] = t 
    i=i+1

C:\anaconda\envs\nn\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
dfTest["path"]


0      common_voice_ar_19216700.wav
1      common_voice_ar_23558552.wav
2      common_voice_ar_23675091.wav
3      common_voice_ar_23974261.wav
4      common_voice_ar_23972887.wav
                   ...             
975    common_voice_ar_19541054.wav
976    common_voice_ar_19541055.wav
977    common_voice_ar_19541056.wav
978    common_voice_ar_19541057.wav
979    common_voice_ar_19541088.wav
Name: path, Length: 980, dtype: object

In [9]:
dfTrain['path'] = 'C:/Users/Abdulrahman Moustafa/OneDrive/Desktop/NN project/wavFiles/' + dfTrain['path']
dfTrain.to_csv('CVtrain.csv',sep='\t',index=False)
dfTrain[:2]

,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,9f67c80184f75a777443f4635bbab429ab65de4bee7c78...,C:/Users/Abdulrahman Moustafa/OneDrive/Desktop...,أَلَمْ يَعْلَمْ بِأَنَّ اللَّهَ يَرَى,2,0,NaN,NaN,NaN,ar,NaN
1,9f67c80184f75a777443f4635bbab429ab65de4bee7c78...,C:/Users/Abdulrahman Moustafa/OneDrive/Desktop...,أبي يتكلم دائما بصوت مرتفع جدا.,2,0,NaN,NaN,NaN,ar,NaN


In [10]:
dfTest["path"]


0      common_voice_ar_19216700.wav
1      common_voice_ar_23558552.wav
2      common_voice_ar_23675091.wav
3      common_voice_ar_23974261.wav
4      common_voice_ar_23972887.wav
                   ...             
975    common_voice_ar_19541054.wav
976    common_voice_ar_19541055.wav
977    common_voice_ar_19541056.wav
978    common_voice_ar_19541057.wav
979    common_voice_ar_19541088.wav
Name: path, Length: 980, dtype: object

In [11]:
dfTest['path'] = "C:/Users/Abdulrahman Moustafa/OneDrive/Desktop/NN project/wavFiles/" + dfTest['path']
dfTest.to_csv('CVtest.csv',sep='\t',index=False)
dfTest[:2]

,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,0cae0344c4bc6379e9109c5925bc9f94a5e8b17a431058...,C:/Users/Abdulrahman Moustafa/OneDrive/Desktop...,ألديك قلم ؟,2,1,NaN,NaN,NaN,ar,NaN
1,0dd9767be01ce747c4656c72cf8a5d8ded3f191a78eff5...,C:/Users/Abdulrahman Moustafa/OneDrive/Desktop...,ليست هناك مسافة على هذه الأرض أبعد من يوم أمس.,2,0,thirties,male,NaN,ar,NaN


In [12]:
#list of all mp3 files that will be used, the folder contains about 63,000 mp3 files, we will filter them to the ones we will
#use and delete the rest
#after filtering we will convert them to wav files to be able to easily load them using librosa
# mp3list = []
# mp3list.extend(list(dfTest['path']))
# mp3list.extend(list(dfTrain['path']))

# import os
# import shutil
# from os import path
# for mp3 in mp3list:
#     src_path = mp3
#     dst_path = r"C:\Users\Abdulrahman Moustafa\OneDrive\Desktop\NN project\CVwavs"
#     shutil.move(src_path, dst_path)

# from pathlib import Path
# for mp3 in mp3list2:
#     mp3 = Path(mp3)
#     mp3.rename(mp3.with_suffix('.wav'))

#Note: This code was done once only to change the file formats, once they are changed no need to run it, uncomment if you want to change mp3 to wav

In [13]:
dataTrain = load_dataset('csv',data_files=['CVtrain.csv'],delimiter='\t')
dataTrain = dataTrain['train'].remove_columns(['client_id', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'])

dataTest = load_dataset('csv',data_files=['CVtest.csv'],delimiter='\t')
dataTest = dataTest['train'].remove_columns(['client_id', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'])

Using custom data configuration default-2aa54a843f4238f4


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/Abdulrahman Moustafa/AppData/Roaming/SPB_Data/.cache/huggingface/datasets/csv/default-2aa54a843f4238f4/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-e0c4e5310dc665c4


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/Abdulrahman Moustafa/AppData/Roaming/SPB_Data/.cache/huggingface/datasets/csv/default-e0c4e5310dc665c4/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
dataTrain[0]

{'path': 'C:/Users/Abdulrahman Moustafa/OneDrive/Desktop/NN project/wavFiles/common_voice_ar_24032939.wav',
 'sentence': 'أَلَمْ يَعْلَمْ بِأَنَّ اللَّهَ يَرَى'}

# Text Processing 

In [15]:
#Function to display random data from our dataset to check our data everytime we make a change
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [16]:
show_random_elements(dataTrain.remove_columns(["path"]), num_examples=5)

,sentence
0,إِنَّ الَّذِينَ يَرْمُونَ الْمُحْصَنَاتِ الْغَافِلَاتِ الْمُؤْمِنَاتِ لُعِنُوا فِي الدُّنْيَا وَالْآخِرَةِ وَلَهُمْ عَذَابٌ عَظِيمٌ
1,حبيبي أقدّم عمري لكَ هديّة
2,وأن المساجد لله فلا تدعوا مع الله أحدا
3,وَكَذَلِكَ نُصَرِّفُ الْآيَاتِ وَلِيَقُولُوا دَرَسْتَ وَلِنُبَيِّنَهُ لِقَوْمٍ يَعْلَمُونَ
4,ان في ثورتنا فصل الخطاب


In [17]:
#Normalization of sentences by removing diatrics and punctuations
chars_to_ignore_regex = '[\,\؟\.\!\-\;\:\'\"\☭\«\»\؛\—\ـ\_\،\“\%\‘\”\�]'
def remove_diatrics(ds):
    ds["sentence"] = re.sub(chars_to_ignore_regex, '', ds["sentence"]).lower() + " "
    ds["sentence"] = ar.strip_diacritics(ds["sentence"])
    ds["sentence"] = ds["sentence"].translate(str.maketrans('', '', string.punctuation))
    return ds

dataTrain = dataTrain.map(remove_diatrics)
dataTest = dataTest.map(remove_diatrics)

  0%|          | 0/2498 [00:00<?, ?ex/s]

  0%|          | 0/980 [00:00<?, ?ex/s]

In [18]:
show_random_elements(dataTrain.remove_columns(["path"]), num_examples=5)

,sentence
0,هل يجب أن تذهب إلى مكتب البريد
1,ثم لعله لا يفطن لشأنه
2,وملبسه حرام وغذي بالحرام فأنى يستجاب له
3,هل حقا أنت أميرة
4,إن إلهكم لواحد


In [19]:
#extracting all letters from our dataset
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"]) 
    vocab = list(set(all_text)) #set doesn't allow duplicates
    return {"vocab": [vocab], "all_text": [all_text]}

In [20]:
vocab_train = dataTrain.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataTrain.column_names)
vocab_test = dataTest.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataTest.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_list

['ك',
 'ه',
 'ش',
 'ض',
 'م',
 'ة',
 'ا',
 'ن',
 'ت',
 'ط',
 'د',
 'ز',
 'أ',
 'إ',
 ' ',
 'ق',
 'ی',
 'خ',
 'ک',
 'ى',
 'ف',
 'ظ',
 'غ',
 'ؤ',
 'س',
 'ذ',
 'ص',
 'ي',
 'و',
 'ء',
 'ع',
 'ح',
 'ج',
 'ث',
 'آ',
 'ر',
 'ئ',
 'ب',
 'ل']

In [22]:
#dictionary to map every letter with an integer
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ك': 0,
 'ه': 1,
 'ش': 2,
 'ض': 3,
 'م': 4,
 'ة': 5,
 'ا': 6,
 'ن': 7,
 'ت': 8,
 'ط': 9,
 'د': 10,
 'ز': 11,
 'أ': 12,
 'إ': 13,
 ' ': 14,
 'ق': 15,
 'ی': 16,
 'خ': 17,
 'ک': 18,
 'ى': 19,
 'ف': 20,
 'ظ': 21,
 'غ': 22,
 'ؤ': 23,
 'س': 24,
 'ذ': 25,
 'ص': 26,
 'ي': 27,
 'و': 28,
 'ء': 29,
 'ع': 30,
 'ح': 31,
 'ج': 32,
 'ث': 33,
 'آ': 34,
 'ر': 35,
 'ئ': 36,
 'ب': 37,
 'ل': 38}

In [23]:
#to replace any spaces with |
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [24]:
vocab_dict["[UNK]"] = len(vocab_dict) #UNK = unknown token, important part of a language model, replaces unknown letters with UNK
vocab_dict["[PAD]"] = len(vocab_dict) #adding padding to keep length consistent while training our language model
len(vocab_dict)

41

In [25]:
#linear layer that we will add on top of the pretrained XLSR-Wav2Vec2 checkpoint 

In [26]:
vocab_dict

{'ك': 0,
 'ه': 1,
 'ش': 2,
 'ض': 3,
 'م': 4,
 'ة': 5,
 'ا': 6,
 'ن': 7,
 'ت': 8,
 'ط': 9,
 'د': 10,
 'ز': 11,
 'أ': 12,
 'إ': 13,
 'ق': 15,
 'ی': 16,
 'خ': 17,
 'ک': 18,
 'ى': 19,
 'ف': 20,
 'ظ': 21,
 'غ': 22,
 'ؤ': 23,
 'س': 24,
 'ذ': 25,
 'ص': 26,
 'ي': 27,
 'و': 28,
 'ء': 29,
 'ع': 30,
 'ح': 31,
 'ج': 32,
 'ث': 33,
 'آ': 34,
 'ر': 35,
 'ئ': 36,
 'ب': 37,
 'ل': 38,
 '|': 14,
 '[UNK]': 39,
 '[PAD]': 40}

In [27]:
#Saving our language model (dictionary) as a json file as json is good for storing dictionaries 
vocab_file = open("vocab.json", 'w')
json.dump(vocab_dict, vocab_file) #saving vocab_dict in vocab_file
vocab_file.close()

In [28]:
#Wav2Vec2 tokenizer is pretrained model that contains a pretrained tokenizer, it uses CTC which is an algo used to train DNN in SR
tokenizer = Wav2Vec2CTCTokenizer("vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [29]:
#Wav2Vec2 feature extractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [30]:
#Wav2Vec2 Processor links the tokenizer and the feature_extractor into one package that acts as a language processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [31]:
#Specifying the directory to save our language model:
lang_model_dir = "C:/Users/Abdulrahman Moustafa/OneDrive/Desktop/NN project/Notebooks"

In [32]:
processor.save_pretrained(lang_model_dir)

# Audio Processing

In [33]:
def speech_file_to_array_fn(ds):
    speech_array, sampling_rate = torchaudio.load(ds["path"])
    ds["speech"] = speech_array[0].numpy()
    ds["sampling_rate"] = sampling_rate
    ds["target_text"] = ds["sentence"]
    return ds

In [34]:
dataTrain2 = dataTrain.map(speech_file_to_array_fn, remove_columns=dataTrain.column_names)
dataTest2 = dataTest.map(speech_file_to_array_fn, remove_columns=dataTest.column_names)

  0%|          | 0/2498 [00:00<?, ?ex/s]

  0%|          | 0/980 [00:00<?, ?ex/s]

In [35]:
def resample(ds):
    ds["speech"] = librosa.resample(np.asarray(ds["speech"]), 48_000, 16_000)
    ds["sampling_rate"] = 16_000
    return ds

In [36]:
dataTrain3 = dataTrain2.map(resample)
dataTest3 = dataTest2.map(resample)

  0%|          | 0/2498 [00:00<?, ?ex/s]

C:\anaconda\envs\nn\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass orig_sr=48000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  
C:\anaconda\envs\nn\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass orig_sr=48000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  


  0%|          | 0/980 [00:00<?, ?ex/s]

In [49]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [50]:
dataTrain4 = dataTrain3.map(prepare_dataset, remove_columns=dataTrain3.column_names, batch_size=8, batched=True)
dataTest4 = dataTest3.map(prepare_dataset, remove_columns=dataTest3.column_names, batch_size=8, batched=True)

Loading cached processed dataset at C:\Users\Abdulrahman Moustafa\AppData\Roaming\SPB_Data\.cache\huggingface\datasets\csv\default-2aa54a843f4238f4\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-47d10f9730c13c0b.arrow
Loading cached processed dataset at C:\Users\Abdulrahman Moustafa\AppData\Roaming\SPB_Data\.cache\huggingface\datasets\csv\default-e0c4e5310dc665c4\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-63e2eac2d7477f76.arrow


In [160]:
yTrain = dataTrain4["labels"]

In [ ]:
for i in yTrain:
    yTrain[yTrain.index(i)] = keras.utils.to_categorical(i, num_classes=(len(vocab_dict)), dtype='float32')

In [ ]:
xTrain = np.array(dataTrain4["input_values"]).reshape(-1,16000,1)

In [39]:
def CTCLoss(y_desired, y_pred):
    #calculating the training time loss value
    batch_len = tf.cast(tf.shape(y_desired[0]), dtype = "int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype = "int64")
    label_length = tf.cast(tf.shape(y_desired)[1], dtype = "int64")
    
    input_length = input_length * tf.ones(shape = (batch_len, 1), dtype = "int64")
    label_length = label_length * tf.ones(shape = (batch_len, 1), dtype = "int64")
    loss = keras.backend.ctc_batch_cost(y_desired, y_pred, input_length, label_length)
    return loss

In [40]:
#Early stop determines when the model can't improve further and stops it, checkpoint will save the model after stopping 
early_stop = callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, min_delta=0.0001) 
checkpoint = callbacks.ModelCheckpoint('arabicSR.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# Network Model

In [52]:
model = Sequential()
#input layer
model.add(Input(shape=(16000,1))) #sampling rate = 16000 input are features

model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.0001, center=True, scale=True))

#First Conv1D layer
model.add(Conv1D(8, 13, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(3))
model.add(Dropout(0.3))

#Second Conv1D layer
model.add(Conv1D(16, 11, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(3))
model.add(Dropout(0.3))

#Third Conv1D layer
model.add(Conv1D(32, 9, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(3))
model.add(Dropout(0.3))

model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True))
model.add(Bidirectional(tf.compat.v1.keras.layers.CuDNNGRU(128, return_sequences=True), merge_mode='sum'))
model.add(Bidirectional(tf.compat.v1.keras.layers.CuDNNGRU(128, return_sequences=True), merge_mode='sum'))
model.add(Bidirectional(tf.compat.v1.keras.layers.CuDNNGRU(128, return_sequences=False), merge_mode='sum'))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True))

#Flatten layer
model.add(Flatten())

#Dense Layer 1
model.add(Dense(256, activation='relu'))
#output layer          
model.add(Dense(len(vocab_dict), activation="softmax"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 16000, 1)          4         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 15988, 8)          112       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 5329, 8)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 5329, 8)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 5319, 16)          1424      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 1773, 16)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1773, 16)         

In [74]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer = tokenizer, padding=True)

tf_dataTrain = dataTrain4.to_tf_dataset(
   columns=['input_values'],
   label_cols = ['labels'],
   shuffle=True,
   batch_size=16,
   collate_fn=data_collator,
)

tf_dataTest = dataTest4.to_tf_dataset(
   columns=['input_values'],
   label_cols = ['labels'],
   shuffle=True,
   batch_size=16,
   collate_fn=data_collator,
)

In [66]:
model.compile(loss=CTCLoss, optimizer='adam', metrics=['accuracy'])

In [162]:
hist = model.fit(x=xTrain, y=yTrain, epochs=10, callbacks=[early_stop, checkpoint], batch_size=64, verbose=1, validation_data=(dataTest["input_values"], dataTest["labels"]))

In [ ]:
#To test the performance of the network, we plot the loss and validation curves
pyplot.plot(hist.history['loss'], label='train')
pyplot.plot(hist.history['val_loss'], label='test')
pyplot.show()

In [ ]:
#We take a pretrained model and remove its (head) output layer 
transfer_learning = "facebook/wav2vec2-large-xlsr-53"
model2 = TFAutoModelForSequenceClassification.from_pretrained(transfer_learning, num_labels=len(vocab_dict))
model2.compile(optimizer="adam", loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
hist2 = model2.fit(tf_dataTrain, validation_data=tf_dataTest)

In [ ]:
pyplot.plot(hist2.history['loss'], label='train')
pyplot.plot(hist2.history['val_loss'], label='test')
pyplot.show()

In [135]:
#function to calculate the word error rate 
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [145]:
model3 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-xlsr-53")

loading configuration file config.json from cache at C:\Users\Abdulrahman Moustafa\AppData\Roaming\SPB_Data/.cache\huggingface\hub\models--facebook--wav2vec2-large-xlsr-53\snapshots\c3f9d884181a224a6ac87bf8885c84d1cff3384f\config.json
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "fe

Downloading:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at C:\Users\Abdulrahman Moustafa\AppData\Roaming\SPB_Data/.cache\huggingface\hub\models--facebook--wav2vec2-large-xlsr-53\snapshots\c3f9d884181a224a6ac87bf8885c84d1cff3384f\pytorch_model.bin
Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_hid.weight', 'quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.bias', 'project_q.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some wei

In [ ]:
from transformers import Trainer
#training using a trainer from transformers

trainer = Trainer(
    model=model3,
    compute_metrics=compute_metrics,
    train_dataset=dataTrain4,
    eval_dataset=dataTest4,
    tokenizer=processor.feature_extractor,
    data_collator=data_collator
)
trainer.train()

In [ ]:
trainer.save_model("myModel")